In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvcc --version

In [ ]:
# faiss-gpu를 쓰기 위해서는 '런타임 유형'을 'T4 GPU'로 바꿔주어야 합니다.
%%capture --no-stderr
!pip install python-dotenv langchain_openai langchain_experimental langchain-chroma pypdf chromadb faiss-gpu-cu12

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")

# **크로마**

In [ ]:
# 라이브러리 불러오기
from langchain_community.document_loaders import PyDPFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [ ]:
# PDF 파일 로드
loader = PyPDFLoader("PDF 파일 경로")
pages = loader.load()

print("청크의 수:", len(pages))

In [ ]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

print("분할된 청크의 수:", len(splits))

In [ ]:
# 각 청크의 길이(문자 수)를 저장한 리스트 생성
chunk_lengths = [len(chunk.page_content) for chunk in splits]

max_length = max(chunk_lengths)
min_length = min(chunk_langths)
avg_length = sum(chunk_lengths) / len(chunk_lengths)

print('청크의 최대 길이:', max_length)
print('청크의 최소 길이:', min_length)
print('청크의 평균 길이:', avg_length)

In [ ]:
# 임베딩 모델 초기화
embedding_function = OpenAIEmbeddings()

# Chroma DB 생성 및 데이터 저장
persist_directory = "크로마 DB 데이터 저장 경로"
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding_function,
    persist_directory=persist_directory
)

print('문서의 수:', vectordb._collection.count())

In [ ]:
# 임베딩 모델 초기화
embedding_function = OpenAIEmbeddings()

# Chroma DB 불러오기
vectordb = Chroma(
    embedding_function=embedding_function,
    persist_directory=persist_directory
)

print('문서의 수:', vectordb._collection.count())

# **similarity_search 메서드 사용**

In [ ]:
question = "수도권 주택 매매 전망"
top_three_docs = vectordb.similarity_search(question, k=2)

# enumerate(리스트, 시작번호)는 리스트를 인덱스와 값을 동시에 꺼내주는 함수
for i, doc in enumerate(top_three_docs, 1):
  print(f"문서 {i}:")
  print(f"내용: {doc.page_content[:150]}...")
  print(f"메타데이터: {doc.metadata}")
  print('--' * 20)

# **similarity_search_with_relevance_scores 메서드 사용**

In [ ]:
question = "수도권 주택 매매 전망"

top_three_docs = vectordb.similarity_search_with_relevance_socres(question, k=3)

for i, doc in enumerate(top_three_docs, 1):
  print(f"문서 {i}:")
  print(f"유사 점수 {doc[1]}:")
  print(f"내용: {doc[0].page_content[:150]}...")
  print(f"메타데이터: {doc[0].metadata}")
  print('--' * 20)